In [9]:
# import libraries
import torch
import os
import sys
import random
import numpy as np
import torch.nn as nn

In [2]:
# import modules
import encoding as enc
import model
import functions as func

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('There are %d GPU(s) avalable.' % torch.cuda.device_count())
else:
    print('No GPUs available. Using CPU instead.')
    device = torch.device('cpu')

No GPUs available. Using CPU instead.


In [9]:
#-------- Seeds --------#
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [12]:
#-------- Parameters --------#

# Just an example below, we can always change later
EPOCHS = 1000
MINI_BATCH_SIZE = 512
LEARNING_RATE = 0.0001
PATIENCE = EPOCHS // 10
N_HIDDEN_NEURONS = 128

# Can be converted to BCE or another
criterion = nn.MSELoss()

DATADIR = "/data/"
MATRICES = "/data/Matrices"


In [14]:
try:
    if len(os.listdir('../data/train')) != 0:
        print("data directories already exists.")
except:
    !unzip ../data/train.zip -d ../data/train
    !unzip ../data/validation.zip -d ../data/validation

Archive:  ../data/train.zip
  inflating: ../data/train/P2_labels.npz  
   creating: ../data/train/__MACOSX/
  inflating: ../data/train/__MACOSX/._P2_labels.npz  
  inflating: ../data/train/P3_input.npz  
  inflating: ../data/train/__MACOSX/._P3_input.npz  
  inflating: ../data/train/P4_input.npz  
  inflating: ../data/train/__MACOSX/._P4_input.npz  
  inflating: ../data/train/P2_input.npz  
  inflating: ../data/train/__MACOSX/._P2_input.npz  
  inflating: ../data/train/P1_input.npz  
  inflating: ../data/train/__MACOSX/._P1_input.npz  
  inflating: ../data/train/P3_labels.npz  
  inflating: ../data/train/__MACOSX/._P3_labels.npz  
  inflating: ../data/train/P4_labels.npz  
  inflating: ../data/train/__MACOSX/._P4_labels.npz  
  inflating: ../data/train/P1_labels.npz  
  inflating: ../data/train/__MACOSX/._P1_labels.npz  
Archive:  ../data/validation.zip
  inflating: ../data/validation/P4_input.npz  
   creating: ../data/validation/__MACOSX/
  inflating: ../data/validation/__MACOSX/._P4

In [22]:
print("train:", os.listdir('../data/train'), '\n')
print("validation:",os.listdir('../data/validation'))

train: ['P2_labels.npz', 'P3_input.npz', 'P4_input.npz', 'P2_input.npz', '__MACOSX', 'P1_input.npz', 'P3_labels.npz', 'P4_labels.npz', 'P1_labels.npz'] 

validation: ['P4_input.npz', '__MACOSX', 'P4_labels.npz']


In [30]:
#-------- Import Dataset --------#

data_list = []
target_list = []

import glob
for fp in glob.glob("../data/train/*input.npz"):
    data = np.load(fp)["arr_0"]
    targets = np.load(fp.replace("input", "labels"))["arr_0"]
    data_list.append(data)
    target_list.append(targets)
    
print(len(data_list))
print(len(target_list))

4
4


In [ ]:
#stats